### Connect to SAS

In [ ]:
###################
### Credentials ###
###################

import keyring
import runpy
import os
import urllib3
urllib3.disable_warnings()

from password_poc import hostname, wd
runpy.run_path(path_name='password_poc.py')
username = keyring.get_password('cas', 'username')
password = keyring.get_password('cas', username)

###################
### Environment ###
###################

import swat

port = 443
os.environ['CAS_CLIENT_SSL_CA_LIST']=str(wd)+str('/ca_cert_poc.pem')
conn =  swat.CAS(hostname, port, username=username, password=password, protocol='http')
print(conn)

### Identify Table

In [20]:
caslib = 'Public'
in_mem_tbl = 'AML_BANK_PREP'

### load table in-memory if not already exists in-memory ###
if conn.table.tableExists(caslib=caslib, name=in_mem_tbl).exists<=0:
    conn.table.loadTable(caslib=caslib, path=str(in_mem_tbl+str('.sashdat')), 
                         casout={'name':in_mem_tbl, 'caslib':caslib, 'promote':True})
    
### show table to verify
conn.table.tableInfo(caslib=caslib, wildIgnore=False, name=in_mem_tbl)

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,AML_BANK_PREP,14302,23,0,utf-8,2022-05-11T13:26:35+00:00,2022-05-11T13:26:35+00:00,2022-06-01T19:28:07+00:00,UTF8,1.967895e+09,0,0,0,AML_BANK_PREP.sashdat,Public,0,sasuser1,,2022-04-22T00:35:21+00:00,1.966207e+09


### Show Columns in Table

In [21]:
### print columns for review of model parameters
conn.table.columnInfo(table=dict(caslib=caslib, name=in_mem_tbl))

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,account_id,,1,double,8,12,,0,0
1,ml_indicator,,2,double,8,12,,0,0
2,checking_only_indicator,,3,double,8,12,,0,0
3,prior_ctr_indicator,,4,double,8,12,,0,0
4,address_change_2x_indicator,,5,double,8,12,,0,0
5,cross_border_trx_indicator,,6,double,8,12,,0,0
6,in_person_contact_indicator,,7,double,8,12,,0,0
7,linkedin_indicator,,8,double,8,12,,0,0
8,atm_deposit_indicator,,9,double,8,12,,0,0
9,trx_10ksum_indicator,,10,double,8,12,,0,0


### Run SAS Data Step

In [22]:
### keep 4 rows
conn.dataStep.runCode(
   code='''
   
        data public.test_data_step;
            set public.aml_bank_prep;
            if account_id < 5;
        run;
        
         ''')

[InputCasTables]

    casLib           Name   Rows  Columns                                    casTable
 0  Public  aml_bank_prep  14302       23  CASTable('aml_bank_prep', caslib='Public')

[OutputCasTables]

    casLib            Name  Rows  Columns  Append Promoted                                     casTable
 0  Public  test_data_step     4       23     NaN        N  CASTable('test_data_step', caslib='Public')

+ Elapsed: 0.0649s, user: 0.109s, sys: 0.0999s, mem: 22.8mb

### Show Table

In [23]:
df = conn.CASTable(caslib=caslib, name='test_data_step')
df.head()

,account_id,ml_indicator,checking_only_indicator,prior_ctr_indicator,address_change_2x_indicator,cross_border_trx_indicator,in_person_contact_indicator,linkedin_indicator,atm_deposit_indicator,trx_10ksum_indicator,primary_transfer_cat,citizenship_country_risk,occupation_risk,credit_score,distance_to_bank,distance_to_employer,income,num_acctbal_chgs_gt2000,num_transactions,analytic_partition
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.710761,1.161250,-0.375869,0.066659,-0.553587,-0.275990,1.0
1,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,-0.571135,-0.895718,-0.518196,0.127209,2.091158,-0.142247,1.0
2,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.029478,-0.290727,0.620418,-0.259672,-0.292378,-0.472146,0.0
3,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,0.198882,0.193265,0.478091,-1.188886,1.340181,-0.115498,1.0


### Run SAS Query

In [31]:
conn.loadactionset('fedSql')

df = conn.fedSql.execDirect(
    query='''
    
    select * from public.aml_bank_prep
    where account_id <5;
    
    ''')

display(df)

NOTE: Added action set 'fedSql'.


,account_id,ml_indicator,checking_only_indicator,prior_ctr_indicator,address_change_2x_indicator,cross_border_trx_indicator,in_person_contact_indicator,linkedin_indicator,atm_deposit_indicator,trx_10ksum_indicator,primary_transfer_cat,citizenship_country_risk,occupation_risk,credit_score,distance_to_bank,distance_to_employer,income,num_acctbal_chgs_gt2000,num_transactions,analytic_partition
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.710761,1.161250,-0.375869,0.066659,-0.553587,-0.275990,1.0
1,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,-0.571135,-0.895718,-0.518196,0.127209,2.091158,-0.142247,1.0
2,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.029478,-0.290727,0.620418,-0.259672,-0.292378,-0.472146,0.0
3,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,0.198882,0.193265,0.478091,-1.188886,1.340181,-0.115498,1.0


### Run Python DF

In [33]:
import numpy as np
import pandas as pd

aml_bank_prep = conn.CASTable(caslib=caslib, name=in_mem_tbl)

### keep 4 rows
test_python = aml_bank_prep[aml_bank_prep['account_id'] < 5]

test_python.head()

,account_id,ml_indicator,checking_only_indicator,prior_ctr_indicator,address_change_2x_indicator,cross_border_trx_indicator,in_person_contact_indicator,linkedin_indicator,atm_deposit_indicator,trx_10ksum_indicator,primary_transfer_cat,citizenship_country_risk,occupation_risk,credit_score,distance_to_bank,distance_to_employer,income,num_acctbal_chgs_gt2000,num_transactions,analytic_partition
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.710761,1.161250,-0.375869,0.066659,-0.553587,-0.275990,1.0
1,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,-0.571135,-0.895718,-0.518196,0.127209,2.091158,-0.142247,1.0
2,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.029478,-0.290727,0.620418,-0.259672,-0.292378,-0.472146,0.0
3,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,0.198882,0.193265,0.478091,-1.188886,1.340181,-0.115498,1.0
